In [34]:
from langchain.document_loaders import PyPDFDirectoryLoader # pdf loding
from langchain.text_splitter import RecursiveCharacterTextSplitter # chunking
from langchain.embeddings import HuggingFaceEmbeddings #embedding
from langchain.embeddings import OpenAIEmbeddings
import pinecone #vectrDb
from langchain.chains import RetrievalQA #Query-Response
import os

In [31]:
from dotenv import load_dotenv
env_var = load_dotenv()

In [15]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [16]:
data

[Document(metadata={'source': 'pdfs\\Program_specific_criteria.pdf', 'page': 0}, page_content=' \n \nCRITERIA FOR ACCREDITING  \nCOMPUTING \nPROGRAMS \n \n \nEffective for Reviews During the \n2015-2016 Accreditation Cycle \n \n \nIncorporates all changes \napproved by the \nABET \nBoard of Directors \n as of \nNovember 1, 2014 \n \n \n \n \n \nComputing Accreditation Commission \n \nABET \n415 N. Charles Street \nBaltimore, MD  21201 \n \nTelephone:  410-347-7700 \nFax:  443-552-3644 \nE-mail:  accreditation@abet.org \nWebsite:  www.abet.org \n \n \nCopyright © 2014 by ABET \nPrinted in the United States of America. \n \n  C001 11/7/2014 '),
 Document(metadata={'source': 'pdfs\\Program_specific_criteria.pdf', 'page': 1}, page_content='All rights reserved.  No part of these criteria may be reproduced in any form or by any means without \nwritten permission from the publisher. \n \nPublished by: ABET \n  415 N. Charles Street \n  Baltimore, MD 21201 \n \n \nRequests for further informat

In [17]:
text_spitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 200)
text_chunks = text_spitter.split_documents(data)

In [18]:
len(text_chunks)

53

In [19]:
# embedding creation using HuggingHub  

In [25]:
embedings = OpenAIEmbeddings()
query = "Tell me about Criteria 6."
query_result = embedings.embed_query(query)


In [28]:
len(query_result)

1536

In [29]:
# intialize Pinecone

In [33]:
PINEKONE_API_KEY = os.getenv('PINEKONE_API_KEY')
PINEKONE_API_KEY

'pcsk_2oqXUS_DiZdQ5SaynMfV62VwD3XrVY9ENdZvMbb63StRLXayFqKbzxsMMco7mXiLrsUNRN'

In [45]:
from pinecone import Pinecone
from pinecone import ServerlessSpec

In [46]:
pc = Pinecone(PINEKONE_API_KEY)

In [42]:
#create index
index_name = "sadatest"

pc.create_index(
    name=index_name,
    dimension=1536, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [53]:
# from langchain.vectorstores import Pinecone
# #pc = Pinecone(PINEKONE_API_KEY, text_chunks, embedings)
# #docsearch = pc.from_texts(texts = [t.page_content for t in text_chunks],embedding = embedings, index_name =index_name )


In [65]:
from pinecone import Pinecone
pc = Pinecone(PINEKONE_API_KEY,index_name = index_name,host="https://test-xujav9t.svc.aped-4627-b74a.pinecone.io")

# pc.from_texts([t.page_content for t in text_chunks],embedding = embedings, index_name = index_name)



In [87]:
# from langchain.vectorstores import Pinecone
# 
# docsearch = Pinecone.from_texts(API_KEY = PINEKONE_API_KEY,texts = [t.page_content for t in text_chunks],embedding = embedings, index_name = "test")

In [138]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=PINEKONE_API_KEY)
index_name = "sadatest"
pc.list_indexes()
index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=OpenAIEmbeddings())
id = [str(i) for i in range(0,len(text_chunks))]
vector_store.add_documents(documents=text_chunks, ids=id)

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52']

In [152]:
query = "What is the Genral Criteria?"
docs = vector_store.similarity_search(query)
for res in docs:
    print(f"* {res.page_content} [{res.metadata}]")

* 2015-2016 Criteria for Accrediting Computing Programs 
GENERAL CRITERIA  
 
Criterion 1.  Students 
Student performance must be evaluated.  Student progress must be monitored to foster success in 
attaining student outcomes, thereby enabling graduates to attain program educational  objectives.  
Students must be advised regarding curriculum and career matters. 
 
The program must have and enforce policies for accepting both new and transfer students, awarding [{'page': 4.0, 'source': 'pdfs\\Program_specific_criteria.pdf'}]
* for that commission. 
Program Criteria – The Program Criteria provide discipline-specific accreditation criteria.  Programs 
must show that they satisfy all of the specific Program Criteria implied by the program title.  Any 
overlapping requirements need be satisfied only once. 
 
 
----------------------------- 
 
All programs seeking accreditation from the Computing Accreditation Commission of ABET must 
demonstrate that they satisfy all of the following Gener

In [141]:
len(docs)

4

In [161]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 4, "score_threshold": 0.7},
)

response = retriever.invoke("How many General Criterias are there?")


In [162]:
for res in response:
    print(f"* {res.page_content} ")

* Continuous I mprovement………………………………………………….3 
 Curriculum……………………………………………………………………4 
 Faculty………………………………………………………………………...4 
 Facilities….…………………………………………………………………..4 
 Institutional Support…………………………………………………...….4 
 
PROGRAM CRITERIA………………………………………………………………5 
 Computer  Science……………………………………………………………5 
 Informati on Systems………………………………………………………..6 
 Information  Technology……………………………………………………6 
 
PROPOSED CHANGES TO CRITERIA……………………………………………8 
  
1 
* 2015-2016 Criteria for Accrediting Computing Programs 
TABLE OF CONTENTS  
 
 
INTRODUCTION…………………………………………………………………….2 
 
GENERAL CRITERIA……………………………………………………………….3 
 Students……………………………………………………………………….3 
 Program Educational Objectives………………………………………… 3 
 Student Outcomes…………………………………………………………...3 
 Continuous I mprovement………………………………………………….3 
 Curriculum……………………………………………………………………4 
 Faculty………………………………………………………………………...4 
 Facilities….…………………………………………………………………..4 
* 2015-2016 Criteria for Accrediting Computing Programs 
GENERAL